In [1]:
pwd


u'/Volumes/data/Kaggle/marchmadness'

In [3]:
import pandas as pd

In [3]:
ls

MarchMad_stage1.ipynb             TourneyCompactResults.csv
README.md                         TourneyDetailedResults.csv
RegularSeasonCompactResults.csv   TourneySeeds.csv
RegularSeasonDetailedResults.csv  TourneySlots.csv
Seasons.csv                       sample_submission.csv
Teams.csv


In [4]:
seasons = pd.read_csv("./Seasons.csv", index_col=0)

In [29]:
seasons.head()

,Dayzero,Regionw,Regionx,Regiony,Regionz
Season,,,,,
1985,10/29/1984,East,West,Midwest,Southeast
1986,10/28/1985,East,Midwest,Southeast,West
1987,10/27/1986,East,Southeast,Midwest,West
1988,11/2/1987,East,Midwest,Southeast,West
1989,10/31/1988,East,West,Midwest,Southeast


In [5]:
teams = pd.read_csv("./Teams.csv", index_col=0)

In [31]:
teams.head()

,Team_Name
Team_Id,
1101,Abilene Chr
1102,Air Force
1103,Akron
1104,Alabama
1105,Alabama A&M


In [7]:
print teams[teams['Team_Name']=='Georgia Tech']

            Team_Name
Team_Id              
1210     Georgia Tech


In [9]:
rscr = pd.read_csv("./RegularSeasonCompactResults.csv")

In [10]:
rsdr = pd.read_csv("./RegularSeasonDetailedResults.csv")

In [11]:
tcr = pd.read_csv("./TourneyCompactResults.csv")

In [12]:
tdr = pd.read_csv("./TourneyDetailedResults.csv")

In [13]:
tseeds = pd.read_csv("./TourneySeeds.csv")

In [14]:
tslots = pd.read_csv("./TourneySlots.csv")

In [15]:
ss = pd.read_csv("./sample_submission.csv")

In [17]:
rscr.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [19]:
rscr.tail()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
145284,2016,132,1114,70,1419,50,N,0
145285,2016,132,1163,72,1272,58,N,0
145286,2016,132,1246,82,1401,77,N,1
145287,2016,132,1277,66,1345,62,N,0
145288,2016,132,1386,87,1433,74,N,0


In [22]:
rsdr.head().transpose()

,0,1,2,3,4
Season,2003,2003,2003,2003,2003
Daynum,10,10,11,11,11
Wteam,1104,1272,1266,1296,1400
Wscore,68,70,73,56,77
Lteam,1328,1393,1437,1457,1208
Lscore,62,63,61,50,71
Wloc,N,N,N,N,N
Numot,0,0,0,0,0
Wfgm,27,26,24,18,30
Wfga,58,62,58,38,61


In [20]:
rsdr.tail()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
71236,2016,132,1114,70,1419,50,N,0,26,52,...,18,4,9,6,20,13,13,6,3,12
71237,2016,132,1163,72,1272,58,N,0,22,48,...,16,11,17,5,21,10,6,3,0,20
71238,2016,132,1246,82,1401,77,N,1,28,58,...,23,15,22,17,23,11,13,5,4,20
71239,2016,132,1277,66,1345,62,N,0,25,60,...,15,17,21,5,22,10,5,4,4,14
71240,2016,132,1386,87,1433,74,N,0,35,54,...,29,11,16,12,21,12,9,5,5,21


In [23]:
tseeds.head()

,Season,Seed,Team
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [24]:
tslots.head()

,Season,Slot,Strongseed,Weakseed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12


In [32]:
top_10_reg = teams.join(pd.DataFrame(rscr['Wteam'].value_counts()[:9]), how='right')
top_10_reg.columns = ['Team_Name', 'Total_Wins']
top_10_reg

,Team_Name,Total_Wins
1181,Duke,819
1242,Kansas,804
1246,Kentucky,765
1314,North Carolina,761
1112,Arizona,746
1393,Syracuse,738
1272,Memphis,692
1424,UNLV,685
1257,Louisville,682


In [33]:
bottom_10_reg = teams.join(pd.DataFrame(rscr['Lteam'].value_counts()[0:9]), how='right')
bottom_10_reg.columns = ['Team_Name', 'Total_Losses']
bottom_10_reg

,Team_Name,Total_Losses
1152,Chicago St,660
1271,MD E Shore,660
1341,Prairie View,645
1306,New Hampshire,619
1363,San Jose St,612
1224,Howard,610
1126,Bethune-Cookman,603
1288,Morgan St,596
1212,Grambling,595


In [36]:
ss.head()

,id,pred
0,2013_1103_1107,0.5
1,2013_1103_1112,0.5
2,2013_1103_1125,0.5
3,2013_1103_1129,0.5
4,2013_1103_1137,0.5
